In [60]:
pip install shap

In [1]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import shap
from sklearn.inspection import PartialDependenceDisplay

from skforecast.plot import set_dark_theme
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from category_encoders.ordinal import OrdinalEncoder


from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

from sklearn import metrics
from sklearn.metrics import  r2_score

import statistics
from  datetime import datetime, timedelta

In [1]:
# read data

stock_df = pd.read_csv('D:/PhD_project/data/tidy_data/stock_tsb_zero_remove.csv')

del stock_df['Unnamed: 0']

stock_df.head()

NameError: name 'pd' is not defined

In [3]:
# make the yearmonth column as date format

stock_df['yearmonth'] = pd.to_datetime(stock_df['yearmonth']) + pd.offsets.MonthEnd(0)

stock_df = stock_df.sort_values(by = 'yearmonth')

stock_df.head()

C:\Users\chama\AppData\Local\Temp\ipykernel_21816\1238878267.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stock_df['yearmonth'] = pd.to_datetime(stock_df['yearmonth']) + pd.offsets.MonthEnd(0)


,yearmonth,region,district,site_code,product_type,product_code,stock_distributed,stock_ordered,unique_id,id,zero_per,site_type
0,2016-01-31,ABIDJAN 1-GRANDS PONTS,ADJAME-PLATEAU-ATTECOUBE,C1014,Implant,AS27137,0,0,2016 JanC1014AS27137,C1014AS27137,0.770833,Health Center
29472,2016-01-31,N'ZI-IFOU-MORONOU,BONGOUANOU,C4017,Oral,AS27000,0,0,2016 JanC4017AS27000,C4017AS27000,0.541667,Hospital
6480,2016-01-31,ABIDJAN 2,ABOBO-EST,C1080,Injectable,AS27133,0,0,2016 JanC1080AS27133,C1080AS27133,0.729167,Hospital
29520,2016-01-31,N'ZI-IFOU-MORONOU,BONGOUANOU,C4017,Oral,AS27132,0,0,2016 JanC4017AS27132,C4017AS27132,0.708333,Hospital
6432,2016-01-31,ABIDJAN 2,ABOBO-EST,C1080,Implant,AS27138,0,0,2016 JanC1080AS27138,C1080AS27138,0.791667,Hospital


In [4]:
# feature engineering

# add rolling statistics

# dummy function to count zeros
count_func = lambda x: (x==0).sum()

stock_df['year'] = pd.DatetimeIndex(stock_df['yearmonth']).year
stock_df['month'] = pd.DatetimeIndex(stock_df['yearmonth']).month

number_lags = 4

for lag in range(1, number_lags + 1):
    stock_df['lag_' + str(lag)] = stock_df.groupby(['site_code', 'product_code'])['stock_distributed'].shift(lag).fillna(0)
    

stock_df['lag_rolling_mean_1234'] = stock_df[['lag_1', 'lag_2', 'lag_3', 'lag_4']].mean(axis=1)

stock_df['rolling_max_4'] = stock_df.groupby('id')['stock_distributed'].rolling(4).max().reset_index(level = 0, drop = True).fillna(0)

stock_df['rolling_zero_per_4'] = (stock_df.groupby('id')['stock_distributed'].rolling(4).apply(count_func)/4).reset_index(level = 0, drop = True).fillna(0)

stock_tidy = stock_df.drop(['lag_1', 'lag_2', 'lag_3', 'lag_4', 'unique_id'], axis = 1)

stock_tidy.head(5)

,yearmonth,region,district,site_code,product_type,product_code,stock_distributed,stock_ordered,id,zero_per,site_type,year,month,lag_rolling_mean_1234,rolling_max_4,rolling_zero_per_4
0,2016-01-31,ABIDJAN 1-GRANDS PONTS,ADJAME-PLATEAU-ATTECOUBE,C1014,Implant,AS27137,0,0,C1014AS27137,0.770833,Health Center,2016,1,0.0,0.0,0.0
29472,2016-01-31,N'ZI-IFOU-MORONOU,BONGOUANOU,C4017,Oral,AS27000,0,0,C4017AS27000,0.541667,Hospital,2016,1,0.0,0.0,0.0
6480,2016-01-31,ABIDJAN 2,ABOBO-EST,C1080,Injectable,AS27133,0,0,C1080AS27133,0.729167,Hospital,2016,1,0.0,0.0,0.0
29520,2016-01-31,N'ZI-IFOU-MORONOU,BONGOUANOU,C4017,Oral,AS27132,0,0,C4017AS27132,0.708333,Hospital,2016,1,0.0,0.0,0.0
6432,2016-01-31,ABIDJAN 2,ABOBO-EST,C1080,Implant,AS27138,0,0,C1080AS27138,0.791667,Hospital,2016,1,0.0,0.0,0.0


In [5]:
# categorical encoding

enc = OrdinalEncoder()

stock_tidy[['region', 'district', 'site_code', 'site_type', 'product_type', 'product_code']] = enc.fit_transform(stock_tidy[['region', 'district', 'site_code', 'site_type', 'product_type', 'product_code']])


In [6]:
# create series and exogenous data

series = stock_tidy[['yearmonth', 'id', 'stock_distributed']]

exog = stock_tidy[['id', 'yearmonth', 'region', 'district', 'site_code', 'site_type', 'product_type', 'product_code', 'year', 'month', 'lag_rolling_mean_1234', 'rolling_max_4', 'rolling_zero_per_4']]

In [7]:
# Transform series and exog to dictionaries

series_dict = series_long_to_dict(
    data      = series,
    series_id = 'id',
    index     = 'yearmonth',
    values    = 'stock_distributed',
    freq      = 'M'
)

exog_dict = exog_long_to_dict(
    data      = exog,
    series_id = 'id',
    index     = 'yearmonth',
    freq      = 'M'
)

In [8]:
# Partition data in train and test - Cross validation

end_train_set1 = '2019-07-31'
start_test_set1 = '2019-08-31'
end_test_set1 = '2019-10-31'

series_dict_train_set1 = {k: v.loc[:end_train_set1,] for k, v in series_dict.items()}
exog_dict_train_set1   = {k: v.loc[:end_train_set1,] for k, v in exog_dict.items()}
series_dict_test_set1  = {k: v.loc[start_test_set1:end_test_set1,] for k, v in series_dict.items()}
exog_dict_test_set1    = {k: v.loc[start_test_set1:end_test_set1,] for k, v in exog_dict.items()}


end_train_set2 = '2019-08-31'
start_test_set2 = '2019-09-30'
end_test_set2 = '2019-11-30'

series_dict_train_set2 = {k: v.loc[:end_train_set2,] for k, v in series_dict.items()}
exog_dict_train_set2   = {k: v.loc[:end_train_set2,] for k, v in exog_dict.items()}
series_dict_test_set2  = {k: v.loc[start_test_set2:end_test_set2,] for k, v in series_dict.items()}
exog_dict_test_set2    = {k: v.loc[start_test_set2:end_test_set2,] for k, v in exog_dict.items()}


end_train_set3 = '2019-09-30'
start_test_set3 = '2019-10-31'

series_dict_train_set3 = {k: v.loc[:end_train_set3,] for k, v in series_dict.items()}
exog_dict_train_set3   = {k: v.loc[:end_train_set3,] for k, v in exog_dict.items()}
series_dict_test_set3  = {k: v.loc[start_test_set3:,] for k, v in series_dict.items()}
exog_dict_test_set3    = {k: v.loc[start_test_set3:,] for k, v in exog_dict.items()}

# LGBM

In [21]:
# Fit forecaster - CV 1

regressor_lgbm = LGBMRegressor(
                boosting_type = 'gbdt',
                metric = 'mae',
                learning_rate = 0.1,
                num_iterations = 200,
                n_estimators = 100,
               #device_type = 'gpu',
                objective = 'poisson')
                #max_bins = 128)

forecaster_lgbm = ForecasterAutoregMultiSeries(
                 regressor          = regressor_lgbm, 
                 transformer_series = None,
                 lags               = 4,  
                 dropna_from_series = False
             )

forecaster_lgbm.fit(series=series_dict_train_set1, exog=exog_dict_train_set1, suppress_warnings=True)

forecaster_lgbm

C:\Users\chama\anaconda3\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005790 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2530
[LightGBM] [Info] Number of data points in the train set: 29445, number of used features: 16
[LightGBM] [Info] Start training from score 2.824852


ForecasterAutoregMultiSeries 
Regressor: LGBMRegressor(metric='mae', num_iterations=200, objective='poisson') 
Lags: [1 2 3 4] 
Transformer for series: None 
Transformer for exog: None 
Series encoding: ordinal_category 
Window size: 4 
Series levels (names): C1004AS27000, C1004AS27132, C1004AS27133, C1004AS27134, C1004AS27137, ... 
Series weights: None 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'dict'> 
Exogenous variables names: region, district, site_code, site_type, product_type, ... 
Training range: 'C1004AS27000': ['2016-01-31', '2019-07-31'], 'C1004AS27132': ['2016-01-31', '2019-07-31'], 'C1004AS27133': ['2016-01-31', '2019-07-31'], ... 
Training index type: DatetimeIndex 
Training index frequency: M 
Regressor parameters: boosting_type: gbdt, class_weight: None, colsample_bytree: 1.0, importance_type: split, learning_rate: 0.1, ... 
fit_kwargs: {} 
Creation date: 2024-08-13 17:03:59 
Last fit date:

In [22]:
# Feature importances
feat_lgbm = forecaster_lgbm.get_feature_importances()

# save file
feat_lgbm.to_csv('D:/PhD_project/data/results/lgbm_feat_imp_skt_cv1.csv', index = False)

In [23]:
# Predict - CV1

boot = 1000

%time predictions_lgbm_cv1 = forecaster_lgbm.predict_bootstrapping(steps=3, exog=exog_dict_test_set1, suppress_warnings=True, n_boot = boot)

CPU times: total: 2h 33min 45s
Wall time: 45min 33s


In [24]:
pred_id = list(predictions_lgbm_cv1.keys())

# Create an empty DataFrame
df = pd.DataFrame(columns=['yearmonth', 'id', 'model', 'horizon'] + [f'X_{i}' for i in range(boot)])
df

,yearmonth,id,model,horizon,X_0,X_1,X_2,X_3,X_4,X_5,...,X_990,X_991,X_992,X_993,X_994,X_995,X_996,X_997,X_998,X_999


In [25]:
for i in pred_id:
    
    df_test = predictions_lgbm_cv1[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'lgbm'
    
    df_test['horizon'] = 1

    df = pd.concat([df, df_test])
    

In [26]:
df

,yearmonth,id,model,horizon,X_0,X_1,X_2,X_3,X_4,X_5,...,X_990,X_991,X_992,X_993,X_994,X_995,X_996,X_997,X_998,X_999
0,2019-08-31,C1004AS27000,lgbm,1,55.737697,55.737697,44.529436,55.737557,60.386740,55.737557,...,49.848546,65.667943,48.742897,55.737670,47.468582,55.737688,67.271663,79.271814,48.742897,68.280610
1,2019-09-30,C1004AS27000,lgbm,1,20.564102,30.695317,49.176086,47.620362,45.846661,37.690117,...,38.228488,23.299680,49.176086,42.339160,69.604920,37.133725,18.516119,36.696247,61.718998,28.155411
2,2019-10-31,C1004AS27000,lgbm,1,19.161960,31.214057,57.563053,36.607284,36.006061,36.607416,...,66.104753,35.269449,38.025341,36.607416,31.384301,36.664094,46.614967,33.748817,37.979912,33.748817
0,2019-08-31,C1004AS27132,lgbm,1,2.033166,2.033166,1.911202,2.033187,-1.424704,2.033187,...,1.779110,3.880100,1.696981,2.033139,1.771066,2.033156,-0.694247,4.665814,1.696981,2.032989
1,2019-09-30,C1004AS27132,lgbm,1,-1.008276,0.153114,0.489299,2.336234,3.618894,0.489299,...,0.227200,0.412862,0.489299,-2.968570,3.199248,0.489329,0.274384,0.735229,0.489123,0.235244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2019-09-30,C5066AS27137,lgbm,1,1.405496,1.638775,2.646878,2.579628,2.587780,2.646878,...,2.646873,2.646873,2.655030,1.740771,2.658223,2.646908,1.405496,2.646873,2.655025,2.646629
2,2019-10-31,C5066AS27137,lgbm,1,1.269726,2.510859,2.511103,2.511129,2.540207,2.511099,...,2.425353,2.511129,1.532083,2.511099,2.511138,2.511099,2.150911,2.511108,2.540181,2.511108
0,2019-08-31,C5066AS27138,lgbm,1,12.408405,12.408405,12.408435,12.408426,12.408435,12.408426,...,12.408405,12.408378,7.838897,12.408378,12.323081,12.408395,12.408405,11.923355,7.838897,12.198088
1,2019-09-30,C5066AS27138,lgbm,1,0.694449,-3.875058,0.694449,0.694423,0.694423,0.694449,...,0.609126,1.229236,0.715273,0.694480,0.977466,0.694480,0.694449,-3.972006,0.504956,0.694449


In [27]:
# update forecaster with new data
forecaster_lgbm.fit(series=series_dict_train_set2, exog=exog_dict_train_set2, suppress_warnings=True)

# Predict - CV2
%time predictions_lgbm_cv2 = forecaster_lgbm.predict_bootstrapping(steps=3, exog=exog_dict_test_set2, suppress_warnings=True, n_boot = boot)

pred_id = list(predictions_lgbm_cv2.keys())

for i in pred_id:
    
    df_test = predictions_lgbm_cv2[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'lgbm'
    
    df_test['horizon'] = 2

    df = pd.concat([df, df_test])

C:\Users\chama\anaconda3\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001896 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2533
[LightGBM] [Info] Number of data points in the train set: 30200, number of used features: 16
[LightGBM] [Info] Start training from score 2.830608
CPU times: total: 1h 37min 48s
Wall time: 28min 51s


In [28]:
# Feature importances
feat_lgbm = forecaster_lgbm.get_feature_importances()

# save file
feat_lgbm.to_csv('D:/PhD_project/data/results/lgbm_feat_imp_skt_cv2.csv', index = False)

In [29]:
# update forecaster with new data
forecaster_lgbm.fit(series=series_dict_train_set3, exog=exog_dict_train_set3, suppress_warnings=True)

# Predict - CV2
%time predictions_lgbm_cv3 = forecaster_lgbm.predict_bootstrapping(steps=3, exog=exog_dict_test_set3, suppress_warnings=True, n_boot = boot)

pred_id = list(predictions_lgbm_cv3.keys())

for i in pred_id:
    
    df_test = predictions_lgbm_cv3[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'lgbm'
    
    df_test['horizon'] = 3

    df = pd.concat([df, df_test])

C:\Users\chama\anaconda3\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000690 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2537
[LightGBM] [Info] Number of data points in the train set: 30955, number of used features: 16
[LightGBM] [Info] Start training from score 2.834193
CPU times: total: 1h 24min 37s
Wall time: 28min 4s


In [30]:
# Feature importances
feat_lgbm = forecaster_lgbm.get_feature_importances()

# save file
feat_lgbm.to_csv('D:/PhD_project/data/results/lgbm_feat_imp_skt_cv3.csv', index = False)

In [31]:
df

,yearmonth,id,model,horizon,X_0,X_1,X_2,X_3,X_4,X_5,...,X_990,X_991,X_992,X_993,X_994,X_995,X_996,X_997,X_998,X_999
0,2019-08-31,C1004AS27000,lgbm,1,55.737697,55.737697,44.529436,55.737557,60.386740,55.737557,...,49.848546,65.667943,48.742897,55.737670,47.468582,55.737688,67.271663,79.271814,48.742897,68.280610
1,2019-09-30,C1004AS27000,lgbm,1,20.564102,30.695317,49.176086,47.620362,45.846661,37.690117,...,38.228488,23.299680,49.176086,42.339160,69.604920,37.133725,18.516119,36.696247,61.718998,28.155411
2,2019-10-31,C1004AS27000,lgbm,1,19.161960,31.214057,57.563053,36.607284,36.006061,36.607416,...,66.104753,35.269449,38.025341,36.607416,31.384301,36.664094,46.614967,33.748817,37.979912,33.748817
0,2019-08-31,C1004AS27132,lgbm,1,2.033166,2.033166,1.911202,2.033187,-1.424704,2.033187,...,1.779110,3.880100,1.696981,2.033139,1.771066,2.033156,-0.694247,4.665814,1.696981,2.032989
1,2019-09-30,C1004AS27132,lgbm,1,-1.008276,0.153114,0.489299,2.336234,3.618894,0.489299,...,0.227200,0.412862,0.489299,-2.968570,3.199248,0.489329,0.274384,0.735229,0.489123,0.235244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2019-11-30,C5066AS27137,lgbm,3,0.371464,0.507324,0.582995,4.233402,4.233402,0.582995,...,0.582993,0.582993,0.582995,0.550911,-0.081501,0.583016,0.371464,0.582993,0.582993,0.905763
2,2019-12-31,C5066AS27137,lgbm,3,0.461291,11.284556,0.672819,0.672842,0.672842,0.672822,...,0.607361,0.672842,0.597150,0.672822,5.827203,0.672822,0.624070,0.672822,0.672819,0.672822
0,2019-10-31,C5066AS27138,lgbm,3,10.459213,10.459213,10.459234,10.459234,10.459213,10.459234,...,10.435112,10.459213,10.181787,10.459213,10.256836,10.459213,10.459213,10.181787,10.045054,1.380206
1,2019-11-30,C5066AS27138,lgbm,3,10.623058,10.208899,10.623058,10.623058,10.623058,10.623058,...,10.420681,10.420681,10.623058,10.623058,10.013776,10.623079,10.623058,1.544051,1.544051,10.598957


In [32]:
# save file

df.to_csv('D:/PhD_project/data/predictions/lgbm_forecasts_skt_cv.csv', index = False)

# XGB

In [33]:
# Fit forecaster

regressor_xgb = XGBRegressor(tree_method = 'hist',
                            objective = 'count:poisson',
                            enable_categorical = True)

forecaster_xgb = ForecasterAutoregMultiSeries(
                 regressor          = regressor_xgb, 
                 transformer_series = None,
                 lags               = 4,  
                 dropna_from_series = False
             )

forecaster_xgb.fit(series=series_dict_train_set1, exog=exog_dict_train_set1, suppress_warnings=True)

forecaster_xgb

ForecasterAutoregMultiSeries 
Regressor: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, objective='count:poisson', ...) 
Lags: [1 2 3 4] 
Transformer for series: None 
Transformer for exog: None 
Series encoding: ordinal_category 
Window size: 4 
Series levels (names): C1004AS27000, C1004AS27132, C1004AS27133, C1004AS27134,

In [34]:
# Feature importances
feat_xgb = forecaster_xgb.get_feature_importances()

# save file
feat_xgb.to_csv('D:/PhD_project/data/results/xgb_feat_imp_skt_cv1.csv', index = False)

In [35]:
# Predict - CV1

%time predictions_xgb_cv1 = forecaster_xgb.predict_bootstrapping(steps=3, exog=exog_dict_test_set1, suppress_warnings=True, n_boot = boot)

CPU times: total: 2h 8min 11s
Wall time: 20min


In [36]:
pred_id = list(predictions_xgb_cv1.keys())

# Create an empty DataFrame
df = pd.DataFrame(columns=['yearmonth', 'id', 'model', 'horizon'] + [f'X_{i}' for i in range(boot)])
df

,yearmonth,id,model,horizon,X_0,X_1,X_2,X_3,X_4,X_5,...,X_990,X_991,X_992,X_993,X_994,X_995,X_996,X_997,X_998,X_999


In [37]:
for i in pred_id:
    
    df_test = predictions_xgb_cv1[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'xgb'
    
    df_test['horizon'] = 1

    df = pd.concat([df, df_test])

In [38]:
# update forecaster with new data
forecaster_xgb.fit(series=series_dict_train_set2, exog=exog_dict_train_set2, suppress_warnings=True)

# Predict - CV2
%time predictions_xgb_cv2 = forecaster_xgb.predict_bootstrapping(steps=3, exog=exog_dict_test_set2, suppress_warnings=True, n_boot = boot)

pred_id = list(predictions_xgb_cv2.keys())

for i in pred_id:
    
    df_test = predictions_xgb_cv2[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'xgb'
    
    df_test['horizon'] = 2

    df = pd.concat([df, df_test])

CPU times: total: 2h 10min 19s
Wall time: 20min 27s


In [39]:
# Feature importances
feat_xgb = forecaster_xgb.get_feature_importances()

# save file
feat_xgb.to_csv('D:/PhD_project/data/results/xgb_feat_imp_skt_cv2.csv', index = False)

In [40]:
# update forecaster with new data
forecaster_xgb.fit(series=series_dict_train_set3, exog=exog_dict_train_set3, suppress_warnings=True)

# Predict - CV3
%time predictions_xgb_cv3 = forecaster_xgb.predict_bootstrapping(steps=3, exog=exog_dict_test_set3, suppress_warnings=True, n_boot = boot)

pred_id = list(predictions_xgb_cv3.keys())

for i in pred_id:
    
    df_test = predictions_xgb_cv3[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'xgb'
    
    df_test['horizon'] = 3

    df = pd.concat([df, df_test])

CPU times: total: 2h 22min 4s
Wall time: 22min 30s


In [41]:
# Feature importances
feat_xgb = forecaster_xgb.get_feature_importances()

# save file
feat_xgb.to_csv('D:/PhD_project/data/results/xgb_feat_imp_skt_cv3.csv', index = False)

In [42]:
# save file

df.to_csv('D:/PhD_project/data/predictions/xgb_forecasts_skt_cv.csv', index = False)

# Random Forest

In [9]:
# Fit forecaster

regressor_rf = RandomForestRegressor(n_estimators=100, max_samples = 0.4)

forecaster_rf = ForecasterAutoregMultiSeries(
                 regressor          = regressor_rf, 
                 transformer_series = None,
                 lags               = 4,  
                 dropna_from_series = False
             )

%time forecaster_rf.fit(series=series_dict_train_set1, exog=exog_dict_train_set1, suppress_warnings=True)

forecaster_rf

CPU times: total: 34.6 s
Wall time: 43.3 s


ForecasterAutoregMultiSeries 
Regressor: RandomForestRegressor(max_samples=0.4) 
Lags: [1 2 3 4] 
Transformer for series: None 
Transformer for exog: None 
Series encoding: ordinal_category 
Window size: 4 
Series levels (names): C1004AS27000, C1004AS27132, C1004AS27133, C1004AS27134, C1004AS27137, ... 
Series weights: None 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'dict'> 
Exogenous variables names: region, district, site_code, site_type, product_type, ... 
Training range: 'C1004AS27000': ['2016-01-31', '2019-07-31'], 'C1004AS27132': ['2016-01-31', '2019-07-31'], 'C1004AS27133': ['2016-01-31', '2019-07-31'], ... 
Training index type: DatetimeIndex 
Training index frequency: M 
Regressor parameters: bootstrap: True, ccp_alpha: 0.0, criterion: squared_error, max_depth: None, max_features: 1.0, ... 
fit_kwargs: {} 
Creation date: 2024-08-14 18:05:28 
Last fit date: 2024-08-14 18:06:09 
Skforecast version: 0

In [10]:
# Feature importances
feat_rf = forecaster_rf.get_feature_importances()

# save file
feat_rf.to_csv('D:/PhD_project/data/results/rf_feat_imp_skt_cv1.csv', index = False)

In [13]:
# Predict - CV1
boot = 1000

%time predictions_rf_cv1 = forecaster_rf.predict_bootstrapping(steps=3, exog=exog_dict_test_set1, suppress_warnings=True, n_boot = boot)

CPU times: total: 5h 24min 24s
Wall time: 5h 42min 50s


In [14]:
pred_id = list(predictions_rf_cv1.keys())

# Create an empty DataFrame
df = pd.DataFrame(columns=['yearmonth', 'id', 'model', 'horizon'] + [f'X_{i}' for i in range(boot)])
df

,yearmonth,id,model,horizon,X_0,X_1,X_2,X_3,X_4,X_5,...,X_990,X_991,X_992,X_993,X_994,X_995,X_996,X_997,X_998,X_999


In [15]:
for i in pred_id:
    
    df_test = predictions_rf_cv1[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'rf'
    
    df_test['horizon'] = 1

    df = pd.concat([df, df_test])

In [16]:
# update forecaster with new data
forecaster_rf.fit(series=series_dict_train_set2, exog=exog_dict_train_set2, suppress_warnings=True)

# Predict - CV2
%time predictions_rf_cv2 = forecaster_rf.predict_bootstrapping(steps=3, exog=exog_dict_test_set2, suppress_warnings=True, n_boot = boot)

pred_id = list(predictions_rf_cv2.keys())

for i in pred_id:
    
    df_test = predictions_rf_cv2[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'rf'
    
    df_test['horizon'] = 2

    df = pd.concat([df, df_test])

CPU times: total: 5h 9min 3s
Wall time: 5h 14min 20s


In [17]:
# Feature importances
feat_rf = forecaster_rf.get_feature_importances()

# save file
feat_rf.to_csv('D:/PhD_project/data/results/rf_feat_imp_skt_cv2.csv', index = False)

In [18]:
# update forecaster with new data
forecaster_rf.fit(series=series_dict_train_set3, exog=exog_dict_train_set3, suppress_warnings=True)

# Predict - CV3
%time predictions_rf_cv3 = forecaster_rf.predict_bootstrapping(steps=3, exog=exog_dict_test_set3, suppress_warnings=True, n_boot = boot)

pred_id = list(predictions_rf_cv3.keys())

for i in pred_id:
    
    df_test = predictions_rf_cv3[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'rf'
    
    df_test['horizon'] = 3

    df = pd.concat([df, df_test])

CPU times: total: 5h 12min 9s
Wall time: 5h 18min 46s


In [19]:
# Feature importances
feat_rf = forecaster_rf.get_feature_importances()

# save file
feat_rf.to_csv('D:/PhD_project/data/results/rf_feat_imp_skt_cv3.csv', index = False)

In [20]:
# save file

df.to_csv('D:/PhD_project/data/predictions/rf_forecasts_skt_cv.csv', index = False)

# MLR

In [9]:
# Fit forecaster

regressor_lr = LinearRegression()

forecaster_lr = ForecasterAutoregMultiSeries(
                 regressor          = regressor_lr, 
                 transformer_series = None,
                 lags               = 4,  
                 dropna_from_series = False
             )

forecaster_lr.fit(series=series_dict_train_set1, exog=exog_dict_train_set1, suppress_warnings=True)

forecaster_lr

ForecasterAutoregMultiSeries 
Regressor: LinearRegression() 
Lags: [1 2 3 4] 
Transformer for series: None 
Transformer for exog: None 
Series encoding: ordinal_category 
Window size: 4 
Series levels (names): C1004AS27000, C1004AS27132, C1004AS27133, C1004AS27134, C1004AS27137, ... 
Series weights: None 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'dict'> 
Exogenous variables names: region, district, site_code, site_type, product_type, ... 
Training range: 'C1004AS27000': ['2016-01-31', '2019-07-31'], 'C1004AS27132': ['2016-01-31', '2019-07-31'], 'C1004AS27133': ['2016-01-31', '2019-07-31'], ... 
Training index type: DatetimeIndex 
Training index frequency: M 
Regressor parameters: ['copy_X: True', 'fit_intercept: True', 'n_jobs: None', 'positive: False'] 
fit_kwargs: {} 
Creation date: 2024-08-14 08:56:44 
Last fit date: 2024-08-14 08:56:48 
Skforecast version: 0.12.0 
Python version: 3.11.5 
Forecaster id

In [10]:
# Feature importances
feat_lr = forecaster_lr.get_feature_importances()

# save file
feat_lr.to_csv('D:/PhD_project/data/results/lr_feat_imp_skt_cv1.csv', index = False)

In [13]:
# Predict - CV1

boot = 1000

%time predictions_lr_cv1 = forecaster_lr.predict_bootstrapping(steps=3, exog=exog_dict_test_set1, suppress_warnings=True, n_boot = boot)

CPU times: total: 10min 16s
Wall time: 13min 11s


In [14]:
pred_id = list(predictions_lr_cv1.keys())

# Create an empty DataFrame
df = pd.DataFrame(columns=['yearmonth', 'id', 'model', 'horizon'] + [f'X_{i}' for i in range(boot)])
df

,yearmonth,id,model,horizon,X_0,X_1,X_2,X_3,X_4,X_5,...,X_990,X_991,X_992,X_993,X_994,X_995,X_996,X_997,X_998,X_999


In [15]:
for i in pred_id:
    
    df_test = predictions_lr_cv1[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'mlr'
    
    df_test['horizon'] = 1

    df = pd.concat([df, df_test])

In [16]:
# update forecaster with new data
forecaster_lr.fit(series=series_dict_train_set2, exog=exog_dict_train_set2, suppress_warnings=True)

# Predict - CV2
%time predictions_lr_cv2 = forecaster_lr.predict_bootstrapping(steps=3, exog=exog_dict_test_set2, suppress_warnings=True, n_boot = boot)

pred_id = list(predictions_lr_cv2.keys())

for i in pred_id:
    
    df_test = predictions_lr_cv2[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'mlr'
    
    df_test['horizon'] = 2

    df = pd.concat([df, df_test])

CPU times: total: 10min 10s
Wall time: 13min 9s


In [17]:
# Feature importances
feat_lr = forecaster_lr.get_feature_importances()

# save file
feat_lr.to_csv('D:/PhD_project/data/results/lr_feat_imp_skt_cv2.csv', index = False)

In [18]:
# update forecaster with new data
forecaster_lr.fit(series=series_dict_train_set3, exog=exog_dict_train_set3, suppress_warnings=True)

# Predict - CV3
%time predictions_lr_cv3 = forecaster_lr.predict_bootstrapping(steps=3, exog=exog_dict_test_set3, suppress_warnings=True, n_boot = boot)

pred_id = list(predictions_lr_cv3.keys())

for i in pred_id:
    
    df_test = predictions_lr_cv3[i]
    
    df_test = df_test.reset_index()
    
    df_test.columns = ['yearmonth'] + [f'X_{i}' for i in range(boot)]
    
    df_test['id'] = i
    
    df_test['model'] = 'mlr'
    
    df_test['horizon'] = 3

    df = pd.concat([df, df_test])

CPU times: total: 9min 7s
Wall time: 13min 3s


In [19]:
# Feature importances
feat_lr = forecaster_lr.get_feature_importances()

# save file
feat_lr.to_csv('D:/PhD_project/data/results/lr_feat_imp_skt_cv3.csv', index = False)

In [20]:
# save file

df.to_csv('D:/PhD_project/data/predictions/lr_forecasts_skt_cv.csv', index = False)